[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/peteyeh/celltrack/HEAD)

In [1]:
%matplotlib inline

import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np

from math import hypot
from scipy.stats import trim_mean
from sklearn.cluster import DBSCAN

from displaytools import *
from improcessing import *

plt.rcParams.update({'font.size': 2})

# Iteration Logic

In [2]:
# Takes a list of (x, y) coordinates corresponding to points in the cluster.
# This cluster is presumed to be INaccurate, so we trim the outliers.
def compute_approx_centroid(cluster):
    y = int(trim_mean(cluster[:, 0], 0.25))
    x = int(trim_mean(cluster[:, 1], 0.25))
    return x, y

# Takes a list of (x, y) coordinates corresponding to points in the cluster.
# This cluster is presumed to be accurate, so we use all values given.
# Output:
#  - x, y: the centroid
#  - box_x: x distance from the centroid for the bounding box
#  - box_y: y distance from the centroid for the bounding box
def compute_centroid_and_box(cluster):
    x = int(np.mean(cluster[:, 1]))
    size_x = np.max(cluster[:, 1]) - np.min(cluster[:, 1])
    box_x = int(1.2*size_x)

    y = int(np.mean(cluster[:, 0]))
    size_y = np.max(cluster[:, 0]) - np.min(cluster[:, 0])
    box_y = int(1.2*size_y)
    
    return x, y, max(15, box_x), max(15, box_y)

def compute_distance(x1, y1, x2, y2):
    return hypot(x2-x1, y2-y1)

# Returns True if both cells are within each other's bounding boxes.
def is_overlapping(cell1, cell2):
    x1, y1, box_x1, box_y1, merge_events, distance_travelled = cell1
    x2, y2, box_x2, box_y2, merge_events, distance_travelled = cell2
    return x1 > x2-box_x2 and x1 < x2+box_x2 and \
           y1 > y2-box_y2 and y1 < y2+box_y2 and \
           x2 > x1-box_x1 and x2 < x1+box_x1 and \
           y2 > y1-box_y1 and y2 < y1+box_y1

# Overlap centroids if they are within each other's bounding boxes.
# Note that this modifies cells in-place.
def dedup_centroids(image, cells):
    image = apply_impipeline(image)
    num_cells = len(cells)
    cell_overlap = np.identity(num_cells)

    # Establish overlapping cells
    for i in range(num_cells):
        for j in range(i+1, num_cells):
            if is_overlapping(cells[i], cells[j]):
                cell_overlap[i][j] = 1
                cell_overlap[j][i] = 1
    
    for i in range(num_cells):
        if sum(cell_overlap[i]) == 1:  # cell is alone
            continue
        else:
            c = np.argwhere(cell_overlap[i])
            cf = c.flatten()
            # Do cool tricks here; establishes if all cells overlapping with
            # cells[i] overlap with each other
            if np.sum(cell_overlap[cf, c]) == sum(cell_overlap[i])**2 and \
               np.sum(np.delete(cell_overlap[cf], cf, axis=1)) == 0 and \
               np.sum(np.delete(cell_overlap[:, cf], cf, axis=0)) == 0:
                # Remove cells that have been merged away
                ct = np.array(list(filter(lambda a: cells[a][4] != -1, cf)))
                # Nothing left to merge
                if len(ct) <= 1:
                    continue
                # Establish new centroid and bounding box
                new_x = int(np.mean(cells[ct][:,0]))
                new_y = int(np.mean(cells[ct][:,1]))
                new_box_x = max(new_x - min(cells[ct][:,0] - cells[ct][:,2]),
                                max(cells[ct][:,0] + cells[ct][:,2]) - new_x)
                new_box_y = max(new_y - min(cells[ct][:,1] - cells[ct][:,3]),
                                max(cells[ct][:,1] + cells[ct][:,3]) - new_y)
                new_box = get_box(image, new_x, new_y, new_box_x, new_box_y)
                new_box_primary_components = \
                    remove_secondary_components(new_box, threshold=0.2)
                # Only merge if it looks like there is only one cell in the new box
                if get_num_components(new_box_primary_components) == 1:
                    # Establish the index of the cell to keep after the merge:
                    #  - Try to keep the cell with the most merge events
                    #  - Failing that, choose the largest cell
                    most_merges = ct[np.where(cells[ct][:,4]==cells[ct][:,4].max())[0]]
                    if len(most_merges) == 1:
                        primary_cell = most_merges[0]
                    else:
                        cell_areas = np.prod(cells[most_merges][:,2:4], axis=1)
                        primary_cell = most_merges[np.argmax(cell_areas)]
#                         primary_cell = most_merges[np.argmax(cells[most_merges][:,5])]
                    cells[primary_cell][0] = new_x
                    cells[primary_cell][1] = new_y
                    cells[primary_cell][2] = new_box_x
                    cells[primary_cell][3] = new_box_y
                    cells[primary_cell][4] += len(ct)-1
                    for ci in ct:
                        if ci != primary_cell:
                            cells[ci][4] = -1
                            cells[ci][5] = -1
                    cell_overlap[cf, c] = np.identity(len(cf))

# Using new_image, update the centroids in cells. Note that this modifies
# cells in-place.
def update_tracking(new_image, cells):
    new_image = apply_impipeline(new_image)
    
    for i in range(len(cells)):
        x, y, box_x, box_y, merge_events, distance_travelled = cells[i]
        
        if merge_events == -1:
            continue
        
        box = get_box(new_image, x, y, box_x, box_y)

        if len(box) == 0:
            cells[i][4] = -1
            cells[i][5] = -1
            continue

        filtered = remove_secondary_components(box)
        num_components = get_num_components(filtered)
        
        # Need to ensure cell is inside the box
        if num_components == 0:
            cells[i][4] = -1
            cells[i][5] = -1
            continue

        cluster = np.argwhere(filtered)
        if num_components == 1:
            sub_x, sub_y, new_box_x, new_box_y = compute_centroid_and_box(cluster)
            cells[i][2] = new_box_x
            cells[i][3] = new_box_y
        else:
            sub_x, sub_y = compute_approx_centroid(cluster)
        new_x = max(x-box_x, 0) + sub_x
        new_y = max(y-box_y, 0) + sub_y
        cells[i][0] = new_x
        cells[i][1] = new_y
        cells[i][5] = distance_travelled + compute_distance(x, y, new_x, new_y)

# Read initial image and find centroids

In [3]:
rfp = cv2.imreadmulti('source_images/RFP-1.tif')[1]

initial_image = rfp[0]

# 0 is a throwaway param, Otsu will determine
threshold, binarized = \
    cv2.threshold(initial_image, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
initial_binarized = np.uint8(initial_image < threshold)

coordinate_bounds = initial_image.shape
print("Image dimensions: " + str(coordinate_bounds))

# display_image_array([initial_image, initial_binarized], columns=2)

Image dimensions: (832, 1128)


In [4]:
cell_points = np.argwhere(1-initial_binarized)
clustering = DBSCAN(eps=6, min_samples=10).fit(cell_points)
labels = clustering.labels_

# Store cells as [[centroid, box_x, box_y], ...]
cells = []
# Track cell metadata
merge_events = 0
distance_travelled = 0

# Note that clustering.labels_ includes -1, which are noisy samples.
# Thus, enforce starting at 0 rather than min(labels).
for i in range(0, max(labels)+1):
    indices = np.where(labels == i)
    cluster = cell_points[indices]
    x, y, box_x, box_y = compute_centroid_and_box(cluster)
    cells += [[x, y, box_x, box_y, merge_events, distance_travelled],]
    
cells = np.array(cells)

print("Tracking %i cells" % len(cells))
display_centroids(initial_image, cells, save_name='0')

gif_buffer = [add_centroids_and_boxes_to_image(initial_image, cells, gif_mode=True)]
# display_centroids(initial_binarized, cells, mode='matplotlib')

Tracking 180 cells


![](saved_images/0.png)

# Proof of Concept for LoG

In [5]:
# image = rfp[1]
# x, y, box_x, box_y, merge_events = cells[2]
# box = get_box(image, x, y, box_x, box_y)

# # LoG
# blurred = cv2.GaussianBlur(np.float64(box), (3,3), 0)
# filtered = cv2.Laplacian(np.float64(blurred), cv2.CV_64F)
# edges = (filtered == 0).astype(int)  # need to threshold this

# display_image_array([box, blurred, filtered, edges], columns=4)

# Proof of Concept for Image Processing Pipeline

In [6]:
# im_canny = apply_canny(rfp[1])
# im_pipeline = apply_impipeline(rfp[1])
# for i in range(10): # len(cells)):
#     x, y, box_x, box_y, merge_events, distance_travelled = cells[i]
#     box_canny = get_box(im_canny, x, y, box_x, box_y)
#     box_pipeline= get_box(im_pipeline, x, y, box_x, box_y)
#     display_image_array([box_canny, remove_secondary_components(box_pipeline)],
#                         columns=2, dpi=150)

# Pipeline sanity check

In [7]:
# def get_sub_images(image, cells):
#     boxes = []
#     for i in range(len(cells)):
#         x, y, box_x, box_y, merge_events, distance_travelled = cells[i]
#         box = get_box(image, x, y, box_x, box_y)
#         boxes += [box,]
#     return boxes

# raw_boxes = get_sub_images(rfp[1], cells)
# processed_boxes = get_sub_images(apply_impipeline(rfp[1]), cells)
# display_image_array(raw_boxes)
# display_image_array([remove_secondary_components(box) for box in processed_boxes])

# Iterate on all images

In [8]:
cumulative_distance = np.zeros((len(cells), len(rfp)))
merge_log = np.zeros((len(cells), len(rfp)))

for i in range(1, len(rfp)):
    image = rfp[i]
    update_tracking(image, cells)
    dedup_centroids(image, cells)
    cumulative_distance[:,i] = cells[:,5]
    merge_log[:,i] = cells[:,4]
    display_centroids(image, cells, save_name=str(i))
    gif_buffer.append(add_centroids_and_boxes_to_image(image, cells, gif_mode=True))

imageio.mimsave('centroids.gif', gif_buffer, duration=0.4)

![](centroids.gif)

# Plot cumulative distance travelled

In [9]:
plt.rcParams.update({'font.size': 8})
plt.figure(dpi=240)
plt.title("Cumulative Distance Travelled")
plt.xlabel("Frame")
plt.ylabel("Pixels")
plt.xlim([0, len(rfp)])
plt.ylim([0, np.max(cumulative_distance)+10])

merge_log_shifted = \
    np.concatenate((np.zeros((len(merge_log), 1)), merge_log[:,:-1]), axis=1)
merge_events = ((merge_log - merge_log_shifted) > 0)

for i in range(len(cumulative_distance)):
    cd = cumulative_distance[i]
    ml = merge_log[i]
    me = merge_events[i]
    
    # Plot cumulative distance
    lost_frame = len(cd)
    if -1 in cd:
        lost_frame = np.where(cd==-1)[0][0]
        cd = cd[:lost_frame]
        ml = ml[:lost_frame]
    num_merges = ml[-1]
    c, l = 'r', 0.1
    if num_merges >= 3:
        c, l = 'g', 0.7
    elif num_merges >= 1:
        c, l = 'b', 0.3
    plt.plot(cd, color=c, linewidth=l)
    
    # Plot merge events
    x = np.argwhere(me==True).flatten()
    y = np.array(cd[x])
    plt.scatter(x, y, s=15, c='k', marker='|', linewidth=0.7)
plt.savefig('distance.png', bbox_inches='tight')
plt.close()

![](distance.png)